## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,51.91,43,0,18.41,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,51.46,46,0,18.41,clear sky
2,2,Vardo,NO,70.3705,31.1107,38.86,76,93,7.47,overcast clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,2.16,53,0,8.72,clear sky
4,4,Hithadhoo,MV,-0.6000,73.0833,82.87,75,100,18.43,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?95


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) &\
                                         (city_data_df['Max Temp']>=min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Hithadhoo,MV,-0.6000,73.0833,82.87,75,100,18.43,overcast clouds
5,5,Nioro,GM,13.3500,-15.7500,74.82,77,100,6.08,overcast clouds
10,10,Bubaque,GW,11.2833,-15.8333,78.87,79,100,5.61,overcast clouds
12,12,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,70.79,83,75,5.75,broken clouds
14,14,Yarada,IN,17.6500,83.2667,79.12,100,20,2.30,mist
17,17,Adrar,MR,20.5022,-10.0711,83.48,16,100,11.88,overcast clouds
18,18,Victoria,HK,22.2855,114.1577,75.33,83,97,1.01,overcast clouds
19,19,Sao Filipe,CV,14.8961,-24.4956,79.86,75,77,10.18,broken clouds
22,22,Talara,PE,-4.5772,-81.2719,75.96,60,0,12.66,clear sky
23,23,Puerto Carreno,CO,6.1890,-67.4859,85.64,68,97,4.12,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             333
City                   333
Country                329
Lat                    333
Lng                    333
Max Temp               333
Humidity               333
Cloudiness             333
Wind Speed             333
Weather Description    333
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df['Country'].replace('',np.nan, inplace = True)
preferred_cities_df.dropna(subset=['Country'], inplace = True)
clean_df = preferred_cities_df

C:\Users\krist\anaconda3\envs\PythonData\lib\site-packages\pandas\core\series.py:4515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\krist\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.87,overcast clouds,-0.6000,73.0833,
5,Nioro,GM,74.82,overcast clouds,13.3500,-15.7500,
10,Bubaque,GW,78.87,overcast clouds,11.2833,-15.8333,
12,Puerto Baquerizo Moreno,EC,70.79,broken clouds,-0.9000,-89.6000,
14,Yarada,IN,79.12,mist,17.6500,83.2667,
17,Adrar,MR,83.48,overcast clouds,20.5022,-10.0711,
18,Victoria,HK,75.33,overcast clouds,22.2855,114.1577,
19,Sao Filipe,CV,79.86,broken clouds,14.8961,-24.4956,
22,Talara,PE,75.96,clear sky,-4.5772,-81.2719,
23,Puerto Carreno,CO,85.64,overcast clouds,6.1890,-67.4859,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels ['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping')

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [14]:
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.87,overcast clouds,-0.6000,73.0833,Scoop Guest House
5,Nioro,GM,74.82,overcast clouds,13.3500,-15.7500,
10,Bubaque,GW,78.87,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
12,Puerto Baquerizo Moreno,EC,70.79,broken clouds,-0.9000,-89.6000,Hotel los Algarrobos
14,Yarada,IN,79.12,mist,17.6500,83.2667,Vishranti
...,...,...,...,...,...,...,...
714,Unai,BR,88.56,overcast clouds,-16.3575,-46.9061,Hotel Regente
717,Pemba,MZ,79.50,broken clouds,-12.9740,40.5178,Guesthouse Samotina
719,Maldonado,UY,83.59,scattered clouds,-34.6667,-54.9167,La Cimarrona
720,Alexandria,EG,73.94,scattered clouds,31.2156,29.9553,Royal Crown Hotel


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('',np.nan, inplace = True)
hotel_df.dropna(subset=['Hotel Name'], inplace = True)
clean_hotel_df = hotel_df

In [19]:
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.87,overcast clouds,-0.6000,73.0833,Scoop Guest House
10,Bubaque,GW,78.87,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
12,Puerto Baquerizo Moreno,EC,70.79,broken clouds,-0.9000,-89.6000,Hotel los Algarrobos
14,Yarada,IN,79.12,mist,17.6500,83.2667,Vishranti
18,Victoria,HK,75.33,overcast clouds,22.2855,114.1577,Mini Hotel Central
...,...,...,...,...,...,...,...
714,Unai,BR,88.56,overcast clouds,-16.3575,-46.9061,Hotel Regente
717,Pemba,MZ,79.50,broken clouds,-12.9740,40.5178,Guesthouse Samotina
719,Maldonado,UY,83.59,scattered clouds,-34.6667,-54.9167,La Cimarrona
720,Alexandria,EG,73.94,scattered clouds,31.2156,29.9553,Royal Crown Hotel


In [21]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""
#<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[['Lat', 'Lng']]
max_temp = clean_hotel_df['Max Temp']
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations,weights = max_temp, dissipating = False,
                                max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))